### Import bibliotek

In [15]:
import json
import os
import requests
from bs4 import BeautifulSoup

### Pobranie ze strony Ceneo.pl opinii o konkretnym produkcie


In [13]:
with open("./headers.json", "r")as jf:
    headers = json.load(jf)



In [22]:
product_id = "10869054"
next_page = f"https://www.ceneo.pl/{product_id}#tab=reviews"
all_reviews = []
while next_page:
    response = requests.get(next_page, headers=headers)
    print(next_page)
    if response.status_code == 200:
        page_dom = BeautifulSoup(response.text, "html.parser")
        reviews = page_dom.select("div.js_product-review:not(.user-post--highlight)")
        print(len(reviews))
        for review in reviews:
            try:
                single_review = {
                    "review_id": review["data-entry-id"],
                    "author": review.select_one("span.user-post__author-name").text.strip(),
                    "recomendation": review.select_one("span.user-post__author-recomendation > em").text.strip(),
                    "stars": review.select_one("span.user-post__score-count").text.strip(),
                    "content": review.select_one("div.user-post__text").text.strip(),
                    "pros": [p.text.strip() for p in review.select("div.review-feature__item--positive")],
                    "cons": [c.text.strip() for c in review.select("div.review-feature__item--negative")],
                    "likes": review.select_one("button.vote-yes > span").text.strip(),
                    "dislikes": review.select_one("button.vote-no > span").text.strip(),
                    "publish_date": review.select_one("span.user-post__published > time:nth-child(1)")['datetime'].strip(),
                    "purchase_date": review.select_one("span.user-post__published > time:nth-child(2)")['datetime'].strip()
                }
                all_reviews.append(single_review)
            except (AttributeError, TypeError):
                pass
        try:
            next_page = "https://www.ceneo.pl" + page_dom.select_one("a.pagination__next")["href"]
        except TypeError:
            next_page = None
print(all_reviews)
        

https://www.ceneo.pl/10869054#tab=reviews
10
https://www.ceneo.pl/10869054/opinie-2
10
https://www.ceneo.pl/10869054/opinie-3
3
[{'review_id': '8212262', 'author': 'Miedziarek', 'recomendation': 'Polecam', 'stars': '4/5', 'content': 'Preparat na chwasty jednoliścienne bardzo dobry, zakupiony głównie na chwastnicę i ze względu na fakt, iż ma nie niszczyć trawy. Na razie spisuje się świetnie.', 'pros': ['jakość', 'skuteczność', 'wydajność'], 'cons': [], 'likes': '0', 'dislikes': '1', 'publish_date': '2018-08-03 21:20:53', 'purchase_date': '2018-07-11 07:46:13'}, {'review_id': '14892935', 'author': 'm...2', 'recomendation': 'Polecam', 'stars': '4,5/5', 'content': 'Działa  zgodnie z opisem. Pozbyłem się problemu chwastnicy na trawniku przy zastosowaniu odpowiedniej dawki.', 'pros': [], 'cons': [], 'likes': '1', 'dislikes': '0', 'publish_date': '2021-08-27 14:14:52', 'purchase_date': '2021-07-30 11:39:32'}, {'review_id': '12497872', 'author': 'a...8', 'recomendation': 'Polecam', 'stars': '4

### Zapisanie wszystkich opinii w bazie danych

In [18]:
if not os.path.exists("./opinions"):
    os.mkdir("./opinions")

In [19]:
with open(f"./opinions/{product_id}.json", "w", encoding = "UTF-8") as jf:
    json.dump(all_reviews, jf, indent=4, ensure_ascii=False)